<a href="https://colab.research.google.com/github/sharathsrini/Behavioral-Clonning/blob/master/31_1_Self_Driving_Car_OG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xvzf cifar-10-python.tar.gz
  
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.1)
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

np.random.seed(68)
tf.set_random_seed(5678)

def tf_log(x): return tf.sigmoid(x)
def d_tf_log(x): return tf_log(x) * (1.0 - tf_log(x))

def tf_Relu(x): return tf.nn.relu(x)
def d_tf_Relu(x): return tf.cast(tf.greater(x,0),dtype=tf.float32)

def tf_softmax(x): return tf.nn.softmax(x)

# Function to unpcicle
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

PathDicom = "./cifar-10-batches-py/"
lstFilesDCM = []  # create an empty list
for dirName, subdirList, fileList in os.walk(PathDicom):
    for filename in fileList:
        if not ".html" in filename.lower() and not  ".meta" in filename.lower():  # check whether the file's DICOM
            lstFilesDCM.append(os.path.join(dirName,filename))

# Read the data traind and Test
batch0 = unpickle(lstFilesDCM[0])
batch1 = unpickle(lstFilesDCM[1])
batch2 = unpickle(lstFilesDCM[2])
batch3 = unpickle(lstFilesDCM[3])
batch4 = unpickle(lstFilesDCM[4])

onehot_encoder = OneHotEncoder(sparse=True)

train_batch = np.vstack((batch0[b'data'],batch1[b'data'],batch2[b'data'],batch3[b'data'],batch4[b'data']))
train_label = np.expand_dims(np.hstack((batch0[b'labels'],batch1[b'labels'],batch2[b'labels'],batch3[b'labels'],batch4[b'labels'])).T,axis=1).astype(np.float32)
train_label = onehot_encoder.fit_transform(train_label).toarray().astype(np.float32)

test_batch = unpickle(lstFilesDCM[5])[b'data']
test_label = np.expand_dims(np.array(unpickle(lstFilesDCM[5])[b'labels']),axis=0).T.astype(np.float32)
test_label = onehot_encoder.fit_transform(test_label).toarray().astype(np.float32)

# Normalize data from 0 to 1
train_batch = (train_batch - train_batch.min(axis=0))/(train_batch.max(axis=0)-train_batch.min(axis=0))
test_batch = (test_batch - test_batch.min(axis=0))/(test_batch.max(axis=0)-test_batch.min(axis=0))

# reshape data
train_batch = np.reshape(train_batch,(len(train_batch),3,32,32))
test_batch = np.reshape(test_batch,(len(test_batch),3,32,32))

# rotate data
train_batch = np.rot90(np.rot90(train_batch,1,axes=(1,3)),3,axes=(1,2)).astype(np.float32)
test_batch = np.rot90(np.rot90(test_batch,1,axes=(1,3)),3,axes=(1,2)).astype(np.float32)

# cnn
class CNNLayer():
    
    def __init__(self,kernel,inchan,outchan):
        self.w = tf.Variable(tf.random_normal([kernel,kernel,inchan,outchan]))
        self.m,self.v = tf.Variable(tf.zeros_like(self.w)), tf.Variable(tf.zeros_like(self.w))
    def getw(self): return self.w
    def feedforward(self,input,stride_num=1):
        self.input = input
        self.layer = tf.nn.conv2d(input,self.w,strides=[1,stride_num,stride_num,1],padding="VALID")
        self.layerA = tf_Relu(self.layer)
        return self.layerA

# fcc
class FCCLayer():
    
    def __init__(self,input,output):
        self.w = tf.Variable(tf.random_normal([input,output]))
        self.m,self.v = tf.Variable(tf.zeros_like(self.w)), tf.Variable(tf.zeros_like(self.w))
    def getw(self): return self.w
    def feedforward(self,input):
        self.input = input
        self.layer = tf.matmul(input,self.w)
        self.layerA = tf_log(self.layer)
        return self.layerA
        
# create layers
l1 = CNNLayer(5,3,24)
l2 = CNNLayer(5,24,36)
l3 = CNNLayer(5,36,48)
l4 = CNNLayer(3,48,64)
l5 = CNNLayer(3,64,64)

l6 = FCCLayer(256,1164)
l7 = FCCLayer(1164,100)
l8 = FCCLayer(100,50)
l9 = FCCLayer(50,10)

weights = [l1.getw(),l2.getw(),
           l3.getw(),l4.getw(),
           l5.getw(),l6.getw(),
           l7.getw(),l8.getw(),
           l9.getw()]

# Hyper Param
learning_rate = 0.001
batch_size = 200
num_epoch = 800
print_size = 50

beta1,beta2 = 0.9,0.999
adam_e = 0.00000001

proportion_rate = 1000
decay_rate = 0.008

# Create graph
x = tf.placeholder(shape=[None,32,32,3],dtype=tf.float32)
y = tf.placeholder(shape=[None,10],dtype=tf.float32)

layer1 = l1.feedforward(x,stride_num=2)
layer2 = l2.feedforward(layer1)
layer3 = l3.feedforward(layer2)

layer4 = l4.feedforward(layer3)
layer5 = l5.feedforward(layer4)

layer6_Input = tf.reshape(layer5,[batch_size,-1])
layer6 = l6.feedforward(layer6_Input)
layer7 = l7.feedforward(layer6)
layer8 = l8.feedforward(layer7)
layer9 = l9.feedforward(layer8)

final_soft = tf_softmax(layer9)

cost = tf.reduce_sum(-1.0 * (y* tf.log(final_soft) + (1-y)*tf.log(1-final_soft)))
correct_prediction = tf.equal(tf.argmax(final_soft, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# auto train
auto_train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost,var_list=weights)

# create Session
with tf.Session(config=config) as sess: 

    sess.run(tf.global_variables_initializer())

    train_total_cost,train_total_acc =0,0
    train_cost_overtime,train_acc_overtime = [],[]

    test_total_cost,test_total_acc = 0,0
    test_cost_overtime,test_acc_overtime = [],[]

    for iter in range(num_epoch):
        
        train_batch,train_label = shuffle(train_batch,train_label)

        # Train Batch
        for current_batch_index in range(0,len(train_batch),batch_size):

            current_batch = train_batch[current_batch_index:current_batch_index+batch_size,:,:,:]
            current_batch_label = train_label[current_batch_index:current_batch_index+batch_size,:]

            sess_results = sess.run( [cost,accuracy,correct_prediction,auto_train], feed_dict= {x:current_batch,y:current_batch_label})
            print("current iter:", iter, " current cost: ", sess_results[0],' current acc: ',sess_results[1], end='\r')
            train_total_cost = train_total_cost + sess_results[0]
            train_total_acc = train_total_acc + sess_results[1]

        # Test batch
        for current_batch_index in range(0,len(test_batch),batch_size):

            current_batch = test_batch[current_batch_index:current_batch_index+batch_size,:,:,:]
            current_batch_label = test_label[current_batch_index:current_batch_index+batch_size,:]

            sess_results = sess.run( [cost,accuracy,correct_prediction], feed_dict= {x:current_batch,y:current_batch_label})
            print("current iter:", iter, " current cost: ", sess_results[0],' current acc: ',sess_results[1], end='\r')
            test_total_cost = test_total_cost + sess_results[0]
            test_total_acc = test_total_acc + sess_results[1]

        # store
        train_cost_overtime.append(train_total_cost/(len(train_batch)/batch_size ) )
        train_acc_overtime.append(train_total_acc/(len(train_batch)/batch_size ) )

        test_cost_overtime.append(test_total_cost/(len(test_batch)/batch_size ) )
        test_acc_overtime.append(test_total_acc/(len(test_batch)/batch_size ) )
        
        # print
        if iter%print_size == 0:
            print('\n=========')
            print("Avg Train Cost: ", train_cost_overtime[-1])
            print("Avg Train Acc: ", train_acc_overtime[-1])
            print("Avg Test Cost: ", test_cost_overtime[-1])
            print("Avg Test Acc: ", test_acc_overtime[-1])
            print('-----------')      
        train_total_cost,train_total_acc,test_total_cost,test_total_acc=0,0,0,0            
                
        

# plot and save
plt.figure()
plt.plot(range(len(train_cost_overtime)),train_cost_overtime,color='y',label='Original Model')
plt.legend()
plt.savefig('og Train Cost over time')

plt.figure()
plt.plot(range(len(train_acc_overtime)),train_acc_overtime,color='y',label='Original Model')
plt.legend()
plt.savefig('og Train Acc over time')

plt.figure()
plt.plot(range(len(test_cost_overtime)),test_cost_overtime,color='y',label='Original Model')
plt.legend()
plt.savefig('og Test Cost over time')

plt.figure()
plt.plot(range(len(test_acc_overtime)),test_acc_overtime,color='y',label='Original Model')
plt.legend()
plt.savefig('og Test Acc over time')


# --- end code --

--2018-03-20 12:42:40--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  1.87MB/s    in 24s     

2018-03-20 12:43:05 (6.64 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1
current iter: 0  current cost:  648.72205  current acc:  0.13
Avg Train Cost:  652.5157741699219
Avg Train Acc:  0.11894000004231929
Avg Test Cost:  649.8954772949219
Avg Test Acc:  0.11

current iter: 50  current cost:  630.4605  current acc:  0.135
Avg Train Cost:  628.6582165527344
Avg Train Acc:  0.16210000005364417
Avg Test Cost:  626.2163586425781
Avg Test Acc:  0.1641000020503998
-----------


current iter: 100  current cost:  629.9233  current acc:  0.155
Avg Train Cost:  626.4286984863281
Avg Train Acc:  0.1738000003695488
Avg Test Cost:  626.1192553710938
Avg Test Acc:  0.1769999998807907
-----------


current iter: 150  current cost:  625.0927  current acc:  0.165
Avg Train Cost:  621.65119921875
Avg Train Acc:  0.19617999961972238
Avg Test Cost:  622.1602697753906
Avg Test Acc:  0.1987999999523163
-----------


current iter: 200  current cost:  624.2868  current acc:  0.195
Avg Train Cost:  620.063732421875
Avg Train Acc:  0.20159999930858613
Avg Test Cost:  620.2621997070313
Avg Test Acc:  0.20399999916553496
-----------


...

current iter: 300  current cost:  622.3759  current acc:  0.22
Avg Train Cost:  618.3032998046875
Avg Train Acc:  0.20643999940156937
Avg Test Cost:  620.0992309570313
Avg Test Acc:  0.2062999990582466
-----------
current iter: 

current iter: 350  current cost:  624.2408  current acc:  0.22
Avg Train Cost:  617.7118842773438
Avg Train Acc:  0.20967999869585038
Avg Test Cost:  619.10751953125
Avg Test Acc:  0.21039999902248382
-----------


current iter: 400  current cost:  622.7089  current acc:  0.17
Avg Train Cost:  617.3322036132812
Avg Train Acc:  0.20865999934077262
Avg Test Cost:  619.6877685546875
Avg Test Acc:  0.20439999908208847
-----------


current iter: 450  current cost:  624.16846  current acc:  0.195
Avg Train Cost:  616.9997983398438
Avg Train Acc:  0.2102999994158745
Avg Test Cost:  620.2354943847656
Avg Test Acc:  0.20640000015497206
-----------


current iter: 500  current cost:  623.77637  current acc:  0.22
Avg Train Cost:  616.5402854003906
Avg Train Acc:  0.21085999935865402
Avg Test Cost:  619.8216235351563
Avg Test Acc:  0.20669999986886978
-----------
